In [2]:
import os

from langchain import hub
from langchain.agents import initialize_agent, Tool, AgentExecutor


from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv
from tavily import TavilyClient

import requests

load_dotenv() 


/Users/aditiphadnis/Documents/Github Projects/SDS-CP022-ai-travel-companion/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


True

In [3]:
from langchain.agents import initialize_agent, Tool, AgentExecutor
from langchain.agents import AgentType
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
import requests

In [20]:
# urls = {
#     'top':'https://www.tripadvisor.com/TravelersChoice-Destinations-cTop-g1',
#     'trending':'https://www.tripadvisor.com/TravelersChoice-Destinations-cTrending-g1',
#     'food':'https://www.tripadvisor.com/TravelersChoice-Destinations-cFood-g1',
#     'culture':'https://www.tripadvisor.com/TravelersChoice-Destinations-cCulture-g1'
# }

urls = [ 'https://www.tripadvisor.com/TravelersChoice-Destinations-cTop-g1', 'https://www.tripadvisor.com/TravelersChoice-Destinations-cTrending-g1']
#     

In [5]:
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

if TAVILY_API_KEY:
    print("Tavily API Key loaded successfully!")
else:
    print("Failed to load Tavily API Key. Please check your .env file.")


Tavily API Key loaded successfully!


In [21]:
# Step 1. Instantiating your TavilyClient
tavily_client = TavilyClient()

# Step 2. Executing a context search query
context = tavily_client.extract(urls = urls)

# Step 3. That's it! You now have a context string that you can feed directly into your RAG Application
print(context)




{'results': [], 'failed_results': [{'url': 'https://www.tripadvisor.com/TravelersChoice-Destinations-cTop-g1', 'error': 'Access denied: Unable to retrieve content from the specified URL'}, {'url': 'https://www.tripadvisor.com/TravelersChoice-Destinations-cTrending-g1', 'error': 'Access denied: Unable to retrieve content from the specified URL'}], 'response_time': 0.11}


In [22]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

In [23]:
# Define a Tavily crawling function
def tavily_crawl_and_extract(url: str) -> str:
    """
    Uses Tavily to crawl the URL and extract information from trip advisor and other websites.
    """
    query= "Top 5 desitinations in India"
    search = tavily_client.get_search_context(max_results=5, topic= "travel", query = query)


    res = search.run(f"{query}")


In [24]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("best travel desitnations from {topic}")

chain = prompt | llm | StrOutputParser() | (lambda output: output.replace("\n", ""))

chain.invoke('India')

'1. Maldives2. Thailand3. Dubai, UAE4. Bhutan5. Singapore6. Bali, Indonesia7. Sri Lanka8. Switzerland9. New Zealand10. Australia'

In [25]:
def travel_search(query):
  for url in urls:
    return tavily_crawl_and_extract(url)

In [26]:
#Define tools for agent
travel_search = Tool(name = "TravelSearch",func= travel_search,description="Use this tool to get the travel information from tavily")
tools = [travel_search]

In [27]:
agent = initialize_agent(tools,llm,agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)


In [28]:
#Define a query
query = "Please tell me travel destinations in and around India"

#Exexute agent
response = agent.run(query)

print(response)  # Inspect the data to find unexpected sets

#print("Agent response :",  response)



> Entering new AgentExecutor chain...
I should use the TravelSearch tool to get information about travel destinations in and around India.
Action: TravelSearch
Action Input: "travel destinations in and around India"

HTTPError: 400 Client Error: Bad Request for url: https://api.tavily.com/search